# Risk calculator process

## Importing modules for running process
In this section all modules which are used by the system are loaded in memory

In [1]:
# Import libraries
import os
import glob
import numpy as np
import pandas as pd

# Import modules
import deforestation as defo
import administrative_level as adle
import livestock_plot as plot
import risk as risk
print("Done!")

Done!


## Setting global parameters
In this section all configurations about folders and files to set up
the system are created. These configurations are global variables for the system

In [2]:
# Global folders
#root_folder = "/home/jovyan/work/BID/dci_fauno/"
root_folder = "D:\\CIAT\\Code\\BID\\dci_fauno"
data_folder = os.path.join(root_folder, "data","risk_calculator")
inputs = os.path.join(data_folder,"inputs")
outputs = os.path.join(data_folder,"outputs")
conf_folder = os.path.join(data_folder,"conf")
# Deforestation folders
def_folder = os.path.join(inputs,"def")
# Administrative level folders
adm_folder = os.path.join(inputs,"adm")
# Livestock plots
plot_folder = os.path.join(inputs,"plot")

# Parameters file
conf_xls = pd.ExcelFile(os.path.join(conf_folder,"conf.xlsx"))
conf = conf_xls.parse("conf")
buffer = conf_xls.parse("buffer")

# Global parameters from conf.xlsx file
glo_crs = int(conf.loc[conf["parameter"] == "glo_crs","value"].values[0])

# Creating folders
# Create folder for deforestation files
if not os.path.exists(def_folder):    
    os.mkdir(def_folder)
# Create folder for administrative level files
if not os.path.exists(adm_folder):    
    os.mkdir(adm_folder)
# Create folder for livestock plot files
if not os.path.exists(plot_folder):    
    os.mkdir(plot_folder)

print("Done!")

Done!


## Downloading data
This section downloads all data require by the system. It keeps all raw files, it will allow to debug if it would be necessary

### Administrative level - Download and unzip
In this section all data about administrative lever of the country will be downloaded and extracted
into specific folders.

In [ ]:
# Getting parameters for downloading adiministrative sources
adm_url_source = conf.loc[conf["parameter"] == "adm_url_source","value"].values[0]
# Download files from source in zip format and extract them
adle.download_data(adm_folder, adm_url_source)
print("Done!")

### Deforestation - Download and unzip
In this section all data about deforestation for a country are downloaded and extracted
into specific folders.

In [ ]:
# Getting parameters for downloading deforestation files
url_base = conf.loc[conf["parameter"] == "def_src_base_url","value"].values[0]
file_name_base = conf.loc[conf["parameter"] == "def_src_file_name","value"].values[0]
def_periods = conf.loc[conf["parameter"] == "def_periods","value"].values[0].split(",")

# Download files from source in zip format and extract them
defo.download_deforestation(def_folder, url_base, file_name_base, def_periods)
print("Done!")

## Fixing data
This chaptar fix the data downloaded into files and formats, in order to can process information

### Administrative level - Reproject
This process reprojects to CRS destiny the original database

In [ ]:
adm_src_adm1_name = conf.loc[conf["parameter"] == "adm_src_adm1_name","value"].values[0]
adm_src_adm1_id = conf.loc[conf["parameter"] == "adm_src_adm1_id","value"].values[0]
adm_src_adm2_name = conf.loc[conf["parameter"] == "adm_src_adm2_name","value"].values[0]
adm_src_adm2_id = conf.loc[conf["parameter"] == "adm_src_adm2_id","value"].values[0]
adm_src_adm3_name = conf.loc[conf["parameter"] == "adm_src_adm3_name","value"].values[0]
adm_src_adm3_id = conf.loc[conf["parameter"] == "adm_src_adm3_id","value"].values[0]
adm_src_area = conf.loc[conf["parameter"] == "adm_src_area","value"].values[0]
adm_src_geometry = conf.loc[conf["parameter"] == "adm_src_geometry","value"].values[0]

# Reporject file
adle.reproject_source(adm_folder, glo_crs,
                      adm_src_adm1_name, adm_src_adm1_id, 
                      adm_src_adm2_name, adm_src_adm2_id, 
                      adm_src_adm3_name, adm_src_adm3_id, 
                      adm_src_area, adm_src_geometry)
print("Done!")

### Deforestation - Extract deforestation values
This section takes the rasters files, which were downloaded, then extract just deforestation pixels
and create a new raster with this data. Finally it creates a summary raster which merge all
others raster files in just one. The output raster files are standarized, it means all pixels are in 
the same size, the CRS is the same for all, and are clipped with the same size

In [3]:
# Getting parameters for filtering deforestation pixels
def_value = int(conf.loc[conf["parameter"] == "def_value","value"].values[0])
def_pixel_size = float(conf.loc[conf["parameter"] == "def_pixel_size","value"].values[0])

print("Extracting!")
defo.extract_deforestation(def_folder, def_value, glo_crs, def_pixel_size)
print("Summaryzing!")
defo.summary_deforestation(def_folder)
print("Done!")

Extracting!
Calculating parameters for new files
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2010_2012\Geotiff\DCCB_SMBYC_CBBQ_V5_2010_2012.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2012_2013\Geotiff\DCCB_SMBYC_CBBQ_V5_2012_2013.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2013_2014\Geotiff\DCCB_SMBYC_CBBQ_V6_2013_2014.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2014_2015\GeoTiff\DCCB_SMBYC_CBBQ_V7_2014_2015.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2015_2016\Geotiff\DCCB_SMBYC_CBBQ_V7_2015_2016.tif
file: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2016_2017\TTTHM4RG3_BEA_T_reprojected_2016_2017.tif
fi

d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2010_2012_tmp.tif
Cropping raster
Dimention: H=61079 W=45525
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2010.tif
Deleting tmp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2010_2012_tmp.tif
Working: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2012_2013\Geotiff\DCCB_SMBYC_CBBQ_V5_2012_2013.tif
Reprojecting: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013_tmp.tif
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013_tmp.tif
Cropping raster
Dimention: H=61079 W=45525
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2012.tif
Deleting tmp: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013

### Deforestation - Parsing to shapefile
In this section the raster files, which were located into rasters folders, are parsed to shapefile format.
Both folders are transformed detail and summary

In [ ]:
# Parsing to shapefile
defo.to_shp(def_folder)
print("Done!")

Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2010.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2010\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2012.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2012\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2013.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\shp_detail\2013\shapefile.shp
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\raster_detail\2014.tif
Transforming
Creating shapefile crs: {'init': 'epsg:3116'}
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\

### Plots
This section compiles all CSV files with information about plots for all years. It clear all records, 
validating that all plots have coordinates and these coordinates are good. Further it sets the region
and locality according to shapefile of reference. Finally it creates a new layer with all data compiled

In [ ]:
# Getting parameters for processing data
plo_character_file = conf.loc[conf["parameter"] == "plo_character_file","value"].values[0]
plo_src_ext_id = conf.loc[conf["parameter"] == "plo_src_ext_id","value"].values[0]
plo_src_lat = conf.loc[conf["parameter"] == "plo_src_lat","value"].values[0]
plo_src_lon = conf.loc[conf["parameter"] == "plo_src_lon","value"].values[0]
plo_src_animals = conf.loc[conf["parameter"] == "plo_src_animals","value"].values[0]
plo_src_adm1 = conf.loc[conf["parameter"] == "plo_src_adm1","value"].values[0]
plo_src_adm2 = conf.loc[conf["parameter"] == "plo_src_adm2","value"].values[0]
plo_src_adm3 = conf.loc[conf["parameter"] == "plo_src_adm3","value"].values[0]
plo_lim_lat_low =  float(conf.loc[conf["parameter"] == "plo_lim_lat_low","value"].values[0])
plo_lim_lat_upp =  float(conf.loc[conf["parameter"] == "plo_lim_lat_upp","value"].values[0])
plo_lim_lon_low =  float(conf.loc[conf["parameter"] == "plo_lim_lon_low","value"].values[0])
plo_lim_lon_upp =  float(conf.loc[conf["parameter"] == "plo_lim_lon_upp","value"].values[0])
# Getting parameters for creating shp
plo_src_crs = int(conf.loc[conf["parameter"] == "plo_src_crs","value"].values[0])

has_adm = (plo_src_adm1 != "") & (plo_src_adm2 != "") & (plo_src_adm3 != "")

# Compiling data
plot.processing_raw_data(plot_folder, plo_character_file, 
              plo_src_ext_id, plo_src_lat, plo_src_lon, plo_src_animals,
              plo_lim_lat_low, plo_lim_lat_upp, plo_lim_lon_low, plo_lim_lon_upp,
              has_adm, plo_src_adm1, plo_src_adm2, plo_src_adm3)
# Fixing data
path_adm_shp = os.path.join(adm_folder,"fixed","adm.shp")
plot.create_data(plot_folder, path_adm_shp, plo_src_crs, glo_crs)

print("Done!")

### Calculating buffer by plot
In this section, the script creates a buffer for all plots. It uses the following ecuation to set radious for the buffer: amount of animals in plot / field capacity. 

In [ ]:
# Creatting buffer
plot.create_buffer(plot_folder, buffer, glo_crs)
print("Done!")

## Analyzing plots

### Calculating deforested areas by plot
In this section the system calculates the proportion of deforested areas by plot. It combines raster deforestation files
and buffer shapefile.

In [ ]:
# Getting parameters for processing data
ris_years = map(str.strip, conf.loc[conf["parameter"] == "ris_years","value"].values[0].split(","))
ris_types_analysis =  map(str.strip, conf.loc[conf["parameter"] == "ris_types_analysis","value"].values[0].split(","))
def_pixel_size = float(conf.loc[conf["parameter"] == "def_pixel_size","value"].values[0])

# De
risk.deforestation_plot(inputs, ris_years, ris_types_analysis, def_pixel_size)